# XAI 2025 TP5 

## PostHoc - Image data

Author: Arthur Babey

Due: 13.05.2025, 23h59

- Professor: Carlos Peña (<a href="mailto:carlos.pena@heig-vd.ch">carlos.pena@heig-vd.ch</a>)
- Assistant 2025: Arthur Babey (<a href="mailto:arthur.babey@heig-vd.ch">arthur.babey@heig-vd.ch</a>)


Date: Spring 2025

* You will need to fill code and answer questions in this notebook

You have a folder called **XAI_TP5_2025_MATERIAL** structured like this :

**XAI_TP5_2025_MATERIAL**/

	data/
		skin-lesion/
            test/
                empty : download it             
            metadata/
                csv with metadata info
            label/
                csv with label info
        
	models/
            model.pth/
                model's weight

            train_metrics.csv

            confusion_matrix.png            
            
            
    XAI_TP5_2025_Image_NOM_PRENOM.ipynb
    
    scripts/
        scripts used for training, evaluation, customdataset and helpers function
            


---
- **Link to download the test set** : https://www.swisstransfer.com/d/1d795359-d947-4c90-8b24-1bba007286b0

- Place it in ./data/skin-lesion


---
# Instructions

During this TP we will use the ISIC 2019 [dataset](https://www.kaggle.com/datasets/andrewmvd/isic-2019). The dataset for ISIC 2019 contains 25,331 images available for the classification of dermoscopic images among nine different diagnostic categories:
- Melanoma
- Melanocytic nevus
- Basal cell carcinoma
- Actinic keratosis
- Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)
- Dermatofibroma
- Vascular lesion
- Squamous cell carcinoma
- None of the above

The dataset was splitted into and train and a test data. The full dataset is accessible on kaggle and the test dataset you will need to use here is stored in ./data/test/

You have been provided with a fine-tuned EfficientNet-B3 model for this dataset. EfficientNet-B3 is a convolutional neural network architecture that offers high classification accuracy while requiring reasonable computing power. It was introduced in the paper 'EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks' by Tan et al. (2019). If you are interested, you can find more details about the architecture in the paper by following this [link](https://arxiv.org/abs/1905.11946).

We also give you multiple function that will make your life easier to plot, make inference, plot on test images. They are described in ./scripts/helpers.py.

---

You will need to complete code, generate plot and answer questions


Goal :

0. Explore : explore the dataset and the model performance
1. Use XAI's method to generate explanation 
2. Expain model's behaviour using PostHoc methods

---

To install the required libraries, run the following commands:

```bash
pip install torch torchvision
pip install captum
pip install efficientnet_pytorch
```

In [1]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

from efficientnet_pytorch import EfficientNet
from captum.attr import LayerGradCam
from torchvision.transforms.functional import to_pil_image

from scripts.evaluate import evaluate
from scripts.helpers import *
from scripts.custom_dataset import CustomImageDataset


/Users/arthurbabey/miniconda3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting paths 

home_path = os.getcwd()

# PATHs 

data_path = os.path.join(home_path, "data", "skin-lesion")
test_dir_path = os.path.join(data_path, "test")
label_test_path = os.path.join(data_path, "label", "label_test.csv")
model_path = os.path.join(home_path, "models")

## 0. Explore dataset and model


- First we load the trained model and the test dataset

- Then we can evaluate it, the run is quite long so you don't have to run it

- The evaluation metrics are shown and a confusion matrix is ploted


In [3]:
# set device cuda, MPS or cpu
device = get_device()

# Load model 
model = EfficientNet.from_pretrained("efficientnet-b3", num_classes=8).to(device)
state = torch.load(os.path.join(model_path, "model.pth"), map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()

# load the test set
test_dataset = CustomImageDataset(csv_path=label_test_path, data_path=test_dir_path, transform=transforms_eval())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# class names
class_names = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]

Loaded pretrained weights for efficientnet-b3


/var/folders/7p/vsz_gnkx7sg_k1blv9xktznr0000gn/T/ipykernel_3008/3517477962.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(os.path.join(model_path, "m

In [4]:
# you do NOT need to rerun this cell
f1_test, acc_test, conf = evaluate(model=model, batch_size = 32)

Accuracy: 0.8489
F1 Score (macro): 0.7252
Confusion Matrix:
[[ 366  175   14   16    0    6    0    4]
 [ 127 1951   15    5    3    8    3    4]
 [  18   27  260   11    0    2    2    9]
 [   6   11   17   41    0    2    0    3]
 [   0    3    0    0  386    0    0    0]
 [   3    4    3    0    0   24    0    1]
 [   1    3    5    0    0    0   27    0]
 [   8    7   13    8    0    3    0   36]]


---
Let's have a look at the confusion matrix : 

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread(os.path.join(model_path, "confusion_matrix.png")))
plt.axis('off')
plt.title("Confusion Matrix")
plt.show()


---
We can have a look at some training metrics 

In [ ]:
# load data
val_loss_df   = pd.read_csv(os.path.join(model_path, "val_loss.csv"))
train_loss_df = pd.read_csv(os.path.join(model_path, "train_loss.csv"))
val_acc_df    = pd.read_csv(os.path.join(model_path, "val_acc.csv"))
train_acc_df  = pd.read_csv(os.path.join(model_path, "train_acc.csv"))

# extract metrics
train_loss = train_loss_df['train_loss'].values
val_loss   = val_loss_df['val_loss'].values
train_acc  = train_acc_df['train_acc'].values
val_acc    = val_acc_df['val_acc'].values
epochs     = range(1, len(train_loss) + 1)

# create figure and axes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

# plot loss
ax1.plot(epochs, train_loss, label='Train Loss', linewidth=2)
ax1.plot(epochs, val_loss,   label='Val Loss',   linewidth=2)
ax1.set_ylabel('Loss')
ax1.set_title('Loss vs. Epoch')
ax1.legend()
ax1.grid(True)

# plot accuracy
ax2.plot(epochs, train_acc, label='Train Acc', linewidth=2)
ax2.plot(epochs, val_acc,   label='Val Acc',   linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy vs. Epoch')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()


### Questions : 

Have a look on the dataset description, it is a balance dataset ? Is there a a class that is way more represented than the other ? if yes wich one ? Assume that the train/test split is random so the frequency in the test set is the same than in the whole dataset

*Answer:*

Identify a potential prediction issue regarding the balance of the dataset by looking at the confusion matrix (be precise).

*Answer:*

Comment the evaluation result, accuracy, f1 score and the confusion matrix ?

*Answer:* 

Comment the learning curves ? the model is overfitting ?

*Answer:* 
   

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /5 </b>
</div>

Commentaires: 

---

Let's have a look at some examples : 

In [ ]:
indices = np.random.randint(0, 1400, size=12)

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
for i, index in enumerate(indices):
    row = i // 4
    col = i % 4
    
    true_label, predicted_label, true_prob, predicted_prob = inference(model, index, test_dataset)       
    img = to_plot(index, test_dataset)
    
    axes[row, col].imshow(img)
    axes[row, col].axis('off')
    if true_prob == predicted_prob:
        color = 'green'
    else:
        color = 'red'
    axes[row, col].set_title(
        f"True label: {true_label} with proba = {true_prob:.2f}\nPred label: {predicted_label} with proba = {predicted_prob:.2f}",
        color=color,
        fontsize=10
    )

plt.tight_layout()
plt.show()

---

Now let's have a look at the model architecture, it is fine tune efficient net model b3 model. In the next cell we can see all the layers name and it's architecture. You can look there to have layer name information that you will need later

In [ ]:
print(model)

### Questions

- **Q1:**. When looking at the images and the model’s predictions, what do you think might cause misclassifications? Can you identify important regions in the images that may affect the model’s decision?

- **Q2:** As you can see, interpreting medical images is not always straightforward. In your own words, why is it important to have domain expertise when analyzing XAI results?

- **Q3:** Can you briefly describe the structure of the model (an efficiennet model)? You don’t need to be exhaustive—just explain the main parts and how the model works in general.



---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /4 </b>
</div>

Commentaires: 

## 1. PostHoc Explanations using Captum


- To make predictions with a neural network, the data input is passed through many layers of multiplication with the learned weights and through non-linear transformations. A single prediction can involve millions of mathematical operations. There is no chance that we humans can follow the exact mapping from data input to prediction.

- Some XAI posthoc methods can help to interpate a neural network result and we will parcours some of them in this section. To do so we will use the [**captum package**](https://captum.ai/)  that put together multiple xai methods including some for vision task for pytorch model. 


### Grad-CAM Exercise

- Grad-CAM is an XAI method that highlights the regions most important to a convolutional neural network’s decision. We provide helper functions to compute and plot Grad-CAM for any input tensor, its label, and a chosen convolutional layer (typically one near the end of the network for best spatial resolution).

In [9]:
def apply_gradcam(model, input_tensor, target_class, layer):
    model.eval()
    gradcam = LayerGradCam(model, layer)
    attr = gradcam.attribute(input_tensor, target=target_class)
    up = F.interpolate(attr, size=input_tensor.shape[2:], mode='bilinear')
    up = up.squeeze().cpu().detach().numpy()
    up = (up - up.min()) / (up.max() - up.min() + 1e-8)  # Normalize to [0, 1]
    return up

def overlay_heatmap(input_tensor, attribution, alpha=0.5):
    img = to_pil_image(input_tensor.squeeze().cpu())

    if torch.is_tensor(attribution):
        hm = attribution.squeeze().cpu().numpy()
    else:
        hm = attribution

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    axes[0].imshow(img)
    axes[0].set_title("Original")
    axes[0].axis("off")
    heatmap = axes[1].imshow(hm, cmap="jet", alpha=alpha)
    axes[1].imshow(img, alpha=1 - alpha)
    axes[1].set_title("Attribution")
    axes[1].axis("off")
    
    # Add color bar
    cbar = fig.colorbar(heatmap, ax=axes[1], fraction=0.046, pad=0.04)
    cbar.set_label("Attribution Intensity", rotation=270, labelpad=15)

    plt.tight_layout()
    plt.show()


Let's see the results on a random images

In [ ]:
idx = 1234 # random index of the test set
img_tensor, class_idx = test_dataset[idx]
img_tensor = img_tensor.unsqueeze(0).to(device)
class_idx = class_idx.argmax().item()

layer = model._conv_head

gc_attr = apply_gradcam(model, img_tensor, class_idx, layer)
overlay_heatmap(img_tensor, gc_attr)

### Grad-CAM Exploration

1. Pick **one image** from the test set (any index you like).

2. Choose:
   - A **target convolutional layer** (usually one near the end of the model).
   - A **target class** to explain (you can use the predicted class or try another one).

3. Run **Grad-CAM** on that image.

4. Then answer:
   - Why did you choose this layer and class?
   - Which parts of the image are highlighted in the heat-map?
   - Why do you think the model focused on those regions?

If you need help checking labels or predictions, you can use these small helpers function:

```python
from scripts.helpers import idx_to_predicted_label, idx_to_true_label, idx_to_proba
``` 

For example : 

In [ ]:
print(idx_to_predicted_label(303, test_dataset, model))

In [ ]:
print(idx_to_true_label(303, test_dataset))

In [ ]:
print(idx_to_proba(303, test_dataset, model))

In [14]:
# YOUR CODE HERE 





*Answers* : 

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /5 </b>
</div>

Commentaires: 

## Saliency and Occlusion

Let's use others explainers from the Captum package, saliency map and occlusion. 

Your task : 

1. **Load the image** at index `836` from the test set.  
   Run the model on it and print the output probabilities.

2. **Answer the following questions**:  
   - What is the **true label** of this image?  
   - What is the **predicted label** by the model?

3. **Use two explanation methods (no Grad-CAM)**:

   Saliency (Vanilla Gradient)
   - This method shows which pixels have the biggest effect on the model's prediction.
   - Brighter areas = more important for the predicted class.
   - **Q1:** Which part of the image is most highlighted in the saliency map?

   Occlusion
   - This method hides small parts of the image with a grey patch and checks how the prediction changes.
   - If hiding a patch drops the score a lot → that patch was important.
   - Use a patch size of `15 x 15` and stride `8`.
   - **Q2:** Are the important areas bigger or smaller than in the saliency map?

    **Plot the results** 

4. **Compare the two methods**
   - **Q3:** Do both explanations highlight the same areas?  
   - **Q4:** Which explanation do you find more useful, and why?


In [15]:
# YOUR CODE HERE





*Answers :*

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: 8 </b>
</div>

Commentaires: 

## 2. Undersand model's behaivour 

### Question – Why does the model confuse MEL and NV?

- From the confusion matrix, we observe that the model frequently confuses **MEL** (melanoma) with **NV** (nevus).  

- Despite having many examples of MEL, it is correctly predicted only about 65% of the time.

- A significant portion of the misclassified examples comes from the confusion between MEL and NV.

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(plt.imread(os.path.join(model_path, "confusion_matrix.png")))
plt.axis('off')
plt.title("Confusion Matrix")
plt.show()


We will try to understand why the model is mixing these two classes using XAI posthoc methods.

You can use the next function to find example of MEL and NEV good and wrong predictions 
---

1. Choose at least **one image from each group** above.
2. Use at least **two different Captum methods** (e.g. GradCAM, Saliency, Occlusion, GuidedGradCam, GradientSHAP).
3. For each image:
   - Show the original image and the explanation map.
   - Look at what parts of the image the model is using.
   - Is the model focusing on the lesion or the background?

4. **Write a short explanation**:
   - What do you think causes the confusion between MEL and NV?
   - Do some lesions look similar?
   - Do some maps highlight the wrong areas?

Try to make a hypothesis from what you observe.

In [17]:
def find_examples(dataset, model, device, seed=42, max_trials=2000):
    import random
    random.seed(seed)
    indices = list(range(len(dataset)))
    random.shuffle(indices)

    matches = {
        "MEL-MEL": None,
        "NV-NV": None,
        "NV-MEL": None,
        "MEL-NV": None
    }

    for idx in indices[:max_trials]:
        img, label = dataset[idx]
        true_name = label_to_names(label)

        model.eval()
        with torch.no_grad():
            logits = model(img.unsqueeze(0).to(device))
            probs = F.softmax(logits, dim=1)
            pred_idx = probs.argmax().item()
            pred_name = class_names[pred_idx]

        key = f"{true_name}-{pred_name}"
        if key in matches and matches[key] is None:
            matches[key] = idx

        if all(v is not None for v in matches.values()):
            break

    return matches


In [18]:
# YOUR CODE HERE





*Answers :*

---
*Réservé pour corrections*
<div class="alert alert-block alert-info">
<b>Corrections: /8 </b>
</div>

Commentaires: 

FIN